# VMAF - Video Multi-Method Assessment Fusion

#### Source: https://github.com/Netflix/vmaf

## Using VMAF with FFmpeg

libvmaf is integrated into FFmpeg, to create complex filters to calculate VMAF on videos with different settings (resolution, codec, fps, etc.). For best practices for calculating VMAF at the right resolution, see the VMAF technical blog [vmaf_resource](https://github.com/Netflix/vmaf_resource/tree/master/python/test/resource).

## Example 1:

This is an example where the distorted video is upscaled to the resolution of the reference video (360p to 1080p). The 'bicubic' interpolation method is used, see more on [FFmpeg](https://www.ffmpeg.org/ffmpeg-scaler.html). 

In [ ]:
! mkdir -p results/vmaf && \
ffmpeg -hide_banner \
    -r 25 -i videos/dist/vp9/bbb_640x360_CRF30.webm \
    -r 25 -i videos/ref/bbb_1920x1080.mp4 \
    -lavfi "[0:v]scale=1920:1080:flags=bicubic,setpts=PTS-STARTPTS[distorted]; \
            [1:v]setpts=PTS-STARTPTS[reference]; \
            [distorted][reference]libvmaf=log_fmt=json:log_path=results/vmaf/bbb_vmaf_scaled.json:model='path=/vmaf/model/vmaf_float_v0.6.1.json'" -f null -

## Example 2:

For UHD (4K) resolution videos, it is recommended to use the model available in the `/vmaf/model/vmaf_float_4k_v0.6.1.json`  directory. A basic instruction is added as an example. Replace the video names with the path where you have loaded your files.

## Graphical representation.

In this section, we plot the VMAF values obtained per frame of the result obtained in Example 1. The arithmetic mean of the obtained values is also shown. 

In [ ]:
import pandas as pd
import json
import plotly.express as px

path_file='results/vmaf/bbb_vmaf_scaled.json'

with open(path_file) as f:
  data = json.load(f)

vmaf_frames=[]

for frame in data['frames']:
    vmaf_frames.append(frame['metrics']['vmaf'])
    
df=pd.DataFrame(vmaf_frames, columns=['VMAF'])

fig = px.line(df,x=df.index, y='VMAF',
              title='VMAF by frame',labels={"index": "Frame"}, template='simple_white', markers=True)

fig.add_hline(y=df.VMAF.mean(), line_width=2, line_dash='dash')

fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center'},
    showlegend=True)

fig.show()

## Using VMAF with Python Library

Finally, the VMAF Python library offers full functionalities from running basic VMAF command lines, software testing, training and validating a new VMAF model on video datasets, data visualization tools, etc. It is the playground to experiment with VMAF.

In [ ]:
# Convert videos to YUV
!ffmpeg -hide_banner -y -i videos/ref/bbb_1920x1080.mp4 -c:v rawvideo -pixel_format yuv420p /tmp/ref.yuv && \
ffmpeg -hide_banner -y -i videos/dist/x264/bbb_1920x1080_CRF23.mp4 -c:v rawvideo -pixel_format yuv420p /tmp/dist.yuv

# Compute VMAF
!echo "Computing VMAF..." && \
cd /vmaf && \
PYTHONPATH=python /vmaf/python/vmaf/script/run_vmaf.py \
  yuv420p 1920 1080 \
  /tmp/ref.yuv \
  /tmp/dist.yuv --out-fmt json > /home/jovyan/work/examples/results/vmaf/bbb_vmaf_python.json 

#Show results
!echo "Results are in file results/vmaf/bbb_vmaf_python.json"